In [1]:
import pandas as pd
import numpy as np
from pydataset import data
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import seaborn as sns
import split_scale
import sklearn
import feature_selection

In [ ]:
#Load the tips dataset.

tips = data('tips')
tips.head()

Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [ ]:
tips['tip_percentage'] = tips.tip/tips.total_bill
tips.head()

Create a column named price_per_person. This should be the total bill divided by the party size.

In [ ]:
tips['price_per_person'] = tips.total_bill/tips.size
tips.head()

Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

Total bill amount and size might be best feature for predicting the tip amount and tip percentage

Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [ ]:
seed = 123

train, test = train_test_split(tips, train_size=.80, random_state=seed)

print(train.shape, test.shape)
train

In [ ]:
numeric_feature_columns = list(train._get_numeric_data().columns)
numeric_feature_columns

In [ ]:
categorical_feature_columns = list((set(train.columns)) - set(train._get_numeric_data().columns))
categorical_feature_columns

In [ ]:
#train = train[numeric_feature_columns]
X_train = train[['total_bill', 'size', 'price_per_person']]
y_train = train['tip']
X_test = test[['total_bill', 'size', 'price_per_person']]
y_test = test['tip']

In [ ]:
test = test[numeric_feature_columns]

In [ ]:
scalar, X_train_scaled, X_test_scaled = split_scale.min_max_scaler(X_train, X_test)

In [ ]:
X_train_scaled

In [ ]:
#X_train = train[['total_bill', 'size', 'price_per_person']]
#y_train = train['tip']

In [ ]:
f_selector = SelectKBest(f_regression, k = 2)
f_selector.fit(X_train_scaled, y_train)
X2 = f_selector.transform(X_train_scaled)

In [ ]:
f_support = f_selector.get_support()
print(f_support)

In [ ]:
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)

In [ ]:
lm = LinearRegression()

In [ ]:
rfe = RFE(lm, 2)

In [ ]:
X_rfe = rfe.fit_transform(X_train_scaled,y_train) 

In [ ]:
mask = rfe.support_
rfe_features = X_train_scaled.loc[:,mask].columns.tolist()
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [ ]:
X_train = train[['total_bill', 'size', 'price_per_person']]
y_train = train['tip_percentage']

In [ ]:
f_selector = SelectKBest(f_regression, k = 2)
f_selector.fit(X_train_scaled, y_train)
X2 = f_selector.transform(X_train_scaled)
f_support = f_selector.get_support()

print(f_support)

In [ ]:
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)

In [ ]:
lm = LinearRegression()

In [ ]:
rfe = RFE(lm, 2)

In [ ]:
X_rfe = rfe.fit_transform(X_train_scaled,y_train)

In [ ]:
mask = rfe.support_
rfe_features = X_train_scaled.loc[:,mask].columns.tolist()
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [ ]:
def select_kbest(X, y, n):
    f_selector = sklearn.feature_selection.SelectKBest(f_regression, k = n)
    f_selector.fit(X, y)
    f_support = f_selector.get_support()
    #f_feature = X.loc[:,f_support].columns.tolist()
    f_feature = X.columns[f_support]
    return f_feature    

In [ ]:
select_kbest(X_train, y_train, 2)

Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [ ]:
def rfe(X, y, n):
    lm = LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n)
    X_rfe = rfe.fit(X,y)
    mask = rfe.support_
    rfe_features = X.columns[mask]
    return rfe_features

In [ ]:
rfe(X_train_scaled, y_train, 2)

# Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [2]:
df = data('swiss')

In [3]:
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [4]:
seed = 123

train, test = train_test_split(df, train_size=.9, random_state=seed)

print(train.shape, test.shape)

(42, 6) (5, 6)


In [ ]:
test

In [5]:
X_train = train[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']]
X_test = test[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']]
y_train = train['Fertility']
y_test = test['Fertility']

In [6]:
feature_selection.select_kbest(X_train, y_train, 3)

Index(['Examination', 'Education', 'Catholic'], dtype='object')

In [7]:
feature_selection.rfe(X_train,y_train,3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')

In [8]:
cols = df[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality', 'Fertility']].corr().nlargest(5, 'Fertility')['Fertility'].index
cols

Index(['Fertility', 'Catholic', 'Infant.Mortality', 'Agriculture',
       'Examination'],
      dtype='object')

In [9]:
cm = df[cols].corr()
cm

,Fertility,Catholic,Infant.Mortality,Agriculture,Examination
Fertility,1.000000,0.463685,0.416556,0.353079,-0.645883
Catholic,0.463685,1.000000,0.175496,0.401095,-0.572742
Infant.Mortality,0.416556,0.175496,1.000000,-0.060859,-0.114022
Agriculture,0.353079,0.401095,-0.060859,1.000000,-0.686542
Examination,-0.645883,-0.572742,-0.114022,-0.686542,1.000000


In [10]:
sns.heatmap(cm, annot = True, cmap = 'viridis')